In [2]:
import re
import pandas as pd

In [3]:
data = pd.read_excel('Downloads/Top Authors_v1.xlsx')
data

Top Prolific Users Social Network User Screen Name  \
0                 Dwight Jason Bain        TWITTER  DwightJasonBain   
1      Elizabeth Hampton (Slattery)        TWITTER       bizzyboo08   
2                      boobstagramm        TWITTER     boobstagramm   
3                             Steve        TWITTER       schism1221   
4                   Polar Cold Caps        TWITTER    PolarColdCaps   
...                             ...            ...              ...   
41083    University of Iowa Surgery        TWITTER    UIowa_Surgery   
41084                 Zachary Costa        TWITTER   ZacharyCosta14   
41085  CallOfDutyChris ‎⍟ 🦃🍁💙💜🇺🇸😈🌻🦄        TWITTER  CallOfDutyEvans   
41086                    Saintister        TWITTER         GSESaint   
41087                    Alex Brown        TWITTER     brownalex275   

                          Full Name  \
0                 Dwight Jason Bain   
1      Elizabeth Hampton (Slattery)   
2                      boobstagramm   
3                             Steve   
4                   Polar Cold Caps   
...                             ...   
41083    University of Iowa Surgery   
41084                 Zachary Costa   
41085  CallOfDutyChris ‎⍟ 🦃🍁💙💜🇺🇸😈🌻🦄   
41086                    Saintister   
41087                    Alex Brown   

                                                     Bio  HCP  \
0      school custodian, Dad, High functioning Autist...  NaN   
1      Unless someone like you cares a whole awful lo...  NaN   
2      Show her breasts on the web is right, show the...  NaN   
3      Dad to 3,\nheart transplant recipient #donatef...  NaN   
4      Keep Your Hair During Chemotherapy. We provide...  NaN   
...                                                  ...  ...   
41083  University of Iowa Department of Surgery striv...  NaN   
41084  I'm 36 ( Autistic Person ) Who Wants #ADateWit...  NaN   
41085  Wife & mother 💖♐️ Dog mom 🐶 Animal lover | Dis...  NaN   
41086  Father, Humanitarian, Musician,Twitch-Affiliat...  NaN   
41087  Sr. Associate - PGS Strategy & Performance Rea...  NaN   

                 Client Profile Lists                 Partner Profile Lists  \
0                                 NaN  Influencers,Inbound Message Profiles   
1                                 NaN                                   NaN   
2      Leqvio_HCPs Bio,Author list_BC                                   NaN   
3                                 NaN              Inbound Message Profiles   
4                                 NaN                                   NaN   
...                               ...                                   ...   
41083                             NaN                                   NaN   
41084                             NaN                                   NaN   
41085                             NaN                                   NaN   
41086                             NaN                                   NaN   
41087                             NaN                                   NaN   

       CF_Profile Phone Number  CF_Profile Email  ...  \
0                          NaN               NaN  ...   
1                          NaN               NaN  ...   
2                          NaN               NaN  ...   
3                          NaN               NaN  ...   
4                          NaN               NaN  ...   
...                        ...               ...  ...   
41083                      NaN               NaN  ...   
41084                      NaN               NaN  ...   
41085                      NaN               NaN  ...   
41086                      NaN               NaN  ...   
41087                      NaN               NaN  ...   

                               Profile Url             Location  \
0      https://twitter.com/DwightJasonBain  Running Springs, CA   
1           https://twitter.com/bizzyboo08        Rochester, NY   
2         https://twitter.com/boobstagramm                  NaN   
3           

In [4]:
pos_list = ["oncology","oncologist", "breast cancer", "oncosurgeon", "surgeon", "surgery", "radonc", "medonc", "#RadOnc", "#MedOnc", "surgical oncologist", "cancer specialist", "oncology fellow","researcher", "scientist", "professor", "chair", "head","oncology", "breast cancer", "surgeon", "cancer"]
neg_list = ["journal", "news", "organization", "clinic", "network", "association", "media", "multimedia", "editor", "college", "society", "education", "CME", "our mission", "retired", "online service"]
titles = ["Dr","Prof","MD","PhD","Phd","FACP","FACS","Onc","doc","onc"]

In [19]:
def extract_hcp_titles_with_names(text):
    # Define a regular expression pattern for HCP titles and names
    hcp_title_name_pattern = r'\b(?:Dr\.|Prof\.|MD|Ph(?:\.D(?:\.)?)?|FACP|FACS|Onc\.|doc)\b'
    # Use re.findall to extract HCP titles with names from the text
    if len(re.findall(hcp_title_name_pattern, text))>0:
        print(re.findall(hcp_title_name_pattern, text))
        

In [21]:
extract_hcp_titles_with_names("DrSGraff")

In [22]:
titles = ['Dr', 'Mr', 'Ms']
input_string = "DrSGraff"

found_titles = [title for title in titles if input_string.find(title) != -1]

if found_titles:
    print(f"Found titles: {', '.join(found_titles)} in the string.")
else:
    print("Did not find any of the titles in the string.")

Found titles: Dr in the string.


In [25]:
!pip install fuzzywuzzy

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [5]:
from fuzzywuzzy import fuzz

def is_any_word_present_fuzzy(words_to_find, input_string, threshold=90):
    words_in_string = input_string.split()

    for word_to_find in words_to_find:
        for existing_word in words_in_string:
            similarity = fuzz.ratio(word_to_find.lower(), existing_word.lower())
            if similarity >= threshold:
                return True

    return False

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
def get_title(inp):
    found_titles = [title for title in titles if inp.find(title) != -1]
    if found_titles:
        return True
    else:
        return False

In [8]:
import numpy as np
data = data.replace(np.nan,"")

In [9]:
name = []
scr=[]
bio=[]
for i in range(len(data)):
    if get_title(data['User Screen Name'].iloc[i]):
        if not is_any_word_present_fuzzy(neg_list, data['Bio'].iloc[i]):
            if is_any_word_present_fuzzy(pos_list, data['Bio'].iloc[i]):
                name.append(data['Full Name'].iloc[i])
                scr.append(data['User Screen Name'].iloc[i])
                bio.append(data['Bio'].iloc[i])

In [11]:
pd.DataFrame(list(zip(scr,name,bio)),columns=["ScreenName","Full Name","Bio"]).to_excel('Downloads/Authors.xlsx')

In [47]:
is_any_word_present_fuzzy(neg_list,"Medical Oncologist, Research Fellow @DanaFarber | @Harvard | PhD @LaStatale | former @IEOufficiale. Interested in breast oncology, HER2, ADCs & good music.")

False

In [14]:
from fuzzywuzzy import fuzz

def is_string_present_fuzzy(substring, sentence, threshold=80):
    similarity = fuzz.ratio(substring.lower(), sentence.lower())
    if (similarity >= threshold):
        print(sentence)
        print(substring)

In [15]:
is_string_present_fuzzy("Kantarjian","DrHKantarjian")

DrHKantarjian
Kantarjian


In [16]:
"Kantarjian" in "DrHKantarjian"

True

### Text Matching

In [1]:
import pandas as pd
data = pd.read_excel('Downloads/Kisqali Veeva List.xlsx')
data

NPI_NUM    MDM_ID BRAND_TIER FIRST_NAME LAST_NAME       SPECIALTY_1  \
0      1245201029   5611909         T4       Adam     Cohen        Hematology   
1      1417945049   4819477         T3  Alexandra    Thomas  Medical Oncology   
2      1366530636   4643480         T4    Allyson     Ocean        Hematology   
3      1891712774   3681666         T4     Cheryl    Duncan               NaN   
4      1558327486   4547956         T4    Daniela     Matei  Medical Oncology   
...           ...       ...        ...        ...       ...               ...   
26234  1922495514  21193814         T3     Baijia     Jiang               NaN   
26235  1932177904   4390133         T4       Suyi      Park               NaN   
26236  1942865001  31988278         T2     Cierra     Omlor               NaN   
26237  1952753873  22999394         T2       Ryan  Holstead               NaN   
26238  1992044374   4623723         T4    Jeffrey    Dunham               NaN   

            SPECIALTY_2 SPECIALTY_3                        JOB_TITLE  \
0      Medical Oncology         NaN  Director, Myeloma Immunotherapy   
1                   NaN         NaN                        Physician   
2      Medical Oncology         NaN                        Physician   
3                   NaN         NaN              Physician Assistant   
4                   NaN         NaN                        Physician   
...                 ...         ...                              ...   
26234               NaN         NaN                              NaN   
26235               NaN         NaN                              NaN   
26236               NaN         NaN              Physician Assistant   
26237               NaN         NaN                              NaN   
26238               NaN         NaN                              NaN   

                                             AFFILIATION  \
0      University of Pennsylvania Medical Center - Ab...   
1      Atrium Health (AH) - Wake Forest Baptist Medic...   
2      NewYork-Presbyterian (NYP) | Weill Cornell Med...   
3      CHRISTUS Shreveport-Bossier Health System - CH...   
4      Northwestern Medicine (NM) - Northwestern Medi...   
...                                                  ...   
26234                              Pmk Medical Group Inc   
26235                       Exodus Behavioral Health Llc   
26236              Suny Healthscience Center at Syracuse   
26237                     Stat Urology Medical Care Pllc   
26238                         Biofit Medical Corporation   

                                        DEPARTMENT    ZIP          CITY STATE  
0                                              NaN  19104  Philadelphia    PA  
1       Hematology and Oncology Services, Clemmons  27012      Clemmons    NC  
2                Gastrointestinal Oncology Service  10021      New York    NY  
3                             Cancer Care Services  71105    Shreveport    LA  
4      Maggie Daley Center for Women's Cancer Care  60611       Chicago    IL  
...                                            ...    ...           ...   ...  
26234                                          NaN  93030        Oxnard    CA  
26235                                          NaN  21230     Baltimore    MD  
26236                                          NaN  13210      Syracuse    NY  
26237                                          NaN  10021      New York    NY  
26238                                          NaN  92262  Palm Springs    CA  

[26239 rows x 14 columns]

In [2]:
#Sprinkler data
data1 = pd.read_excel('Downloads/Sprinklr distinct user list.xlsx')
data1

All Users Social Network  \
0                              The New York Times        TWITTER   
1                                            P!nk        TWITTER   
2                                        Fox News        TWITTER   
3                         The Wall Street Journal        TWITTER   
4                             The Washington Post        TWITTER   
...                                           ...            ...   
56303                               Kelly.Mueller        TWITTER   
56304  🇺🇸 The Hemp Christ (United States King) 🇺🇸        TWITTER   
56305                                    Jeff2001        TWITTER   
56306                                   Zemetrius        TWITTER   
56307                                 El Chato 🐶.        TWITTER   

      User Screen Name                                   Full Name  \
0              nytimes                          The New York Times   
1                 Pink                                        P!nk   
2              FoxNews                                    Fox News   
3                  WSJ                     The Wall Street Journal   
4       washingtonpost                         The Washington Post   
...                ...                                         ...   
56303     Nisar_Ali234                               Kelly.Mueller   
56304    TheHempChrist  🇺🇸 The Hemp Christ (United States King) 🇺🇸   
56305     JeffreyL2137                                    Jeff2001   
56306       Zemetrius_                                   Zemetrius   
56307       Rsalinas81                                 El Chato 🐶.   

      Allocation status                                                Bio  \
0         Not allocated  News tips? Share them here: http://nyti.ms/2FV...   
1         Not allocated  TRUSTFALL Deluxe out now 💜 “Dreaming” with @th...   
2         Not allocated  Follow America's #1 cable news network, delive...   
3         Not allocated  Sign up for our newsletters and alerts: http:/...   
4         Not allocated                         Democracy Dies in Darkness   
...                 ...                                                ...   
56303     Not allocated  Make peace with everything,be at peace with wa...   
56304     Not allocated  Jesus Christ! I saved the United States by fli...   
56305     Not allocated                            Please don't follow me.   
56306     Not allocated  sure could go for a mean tweet and $1.79 gas r...   
56307     Not allocated                      Loves dogs and playing poker.   

                                    Client Profile Lists  \
0                                          PD engagement   
1                                                    NaN   
2                                          PD engagement   
3      Potential PAG list_BC,Authorlist-PD,PD engagement   
4                                          PD engagement   
...                                                  ...   
56303                                                NaN   
56304                                                NaN   
56305                                                NaN   
56306                                                NaN   
56307                                                NaN   

                                   Partner Profile Lists  \
0      Financial Report - International,Verified Prof...   
1                                                    NaN   
2      Verified Profiles,Influencers,Inbound Message ...   
3      Financial Report - International,Verified Prof...   
4      Inbound Message Profiles,Financial Report - In...   
...                                                  ...   
56303                                                NaN   
56304                                                NaN   
56305                                                NaN   
56306                                                NaN   
56307                                                NaN   

   

In [3]:
import numpy as np
data= data.replace(np.nan,"")
data1= data1.replace(np.nan,"")

In [12]:
first_name = []
last_name=[]
middle_name=[]
User_Screen_Name=[]
full_name = []
for i in range(20000,30000):
    print(i)
    for j in range(len(data1)):
        if len(data['FIRST_NAME'].iloc[i])>3:
            if data['FIRST_NAME'].iloc[i] in data1['User Screen Name'].iloc[j]:
                first_name.append(data['FIRST_NAME'].iloc[i])
                last_name.append(data['LAST_NAME'].iloc[i])
#                 middle_name.append(data['Middle Name'].iloc[i])
                User_Screen_Name.append(data1['User Screen Name'].iloc[j])
                full_name.append(data1['Full Name'].iloc[j])
            elif data['FIRST_NAME'].iloc[i] in data1['Full Name'].iloc[j]:
                first_name.append(data['FIRST_NAME'].iloc[i])
                last_name.append(data['LAST_NAME'].iloc[i])
#                 middle_name.append(data['Middle Name'].iloc[i])
                User_Screen_Name.append(data1['User Screen Name'].iloc[j])
                full_name.append(data1['Full Name'].iloc[j])
#             elif len(data['Middle Name'].iloc[i])>3:
#                 if data['Middle Name'].iloc[i] in data1['User Screen Name'].iloc[j]:
#                     first_name.append(data['First Name'].iloc[i])
#                     last_name.append(data['Last Name'].iloc[i])
#                     middle_name.append(data['Middle Name'].iloc[i])
#                     User_Screen_Name.append(data1['User Screen Name'].iloc[j])
#                     full_name.append(data1['Full Name'].iloc[j])  
#                 elif data['Middle Name'].iloc[i] in data1['Full Name'].iloc[j]:
#                     first_name.append(data['First Name'].iloc[i])
#                     last_name.append(data['Last Name'].iloc[i])
#                     middle_name.append(data['Middle Name'].iloc[i])
#                     User_Screen_Name.append(data1['User Screen Name'].iloc[j])
#                     full_name.append(data1['Full Name'].iloc[j])
            elif len(data['LAST_NAME'].iloc[i])>3:
                if data['LAST_NAME'].iloc[i] in data1['Full Name'].iloc[j]:
                    first_name.append(data['FIRST_NAME'].iloc[i])
                    last_name.append(data['LAST_NAME'].iloc[i])
#                     middle_name.append(data['Middle Name'].iloc[i])
                    User_Screen_Name.append(data1['User Screen Name'].iloc[j])
                    full_name.append(data1['Full Name'].iloc[j])   
                elif data['LAST_NAME'].iloc[i] in data1['Full Name'].iloc[j]:
                    first_name.append(data['FIRST_NAME'].iloc[i])
                    last_name.append(data['LAST_NAME'].iloc[i])
#                     middle_name.append(data['Middle Name'].iloc[i])
                    User_Screen_Name.append(data1['User Screen Name'].iloc[j])
                    full_name.append(data1['Full Name'].iloc[j])

20000
20001
20002
20003
20004
20005
20006
20007
20008
20009
20010
20011
20012
20013
20014
20015
20016
20017
20018
20019
20020
20021
20022
20023
20024
20025
20026
20027
20028
20029
20030
20031
20032
20033
20034
20035
20036
20037
20038
20039
20040
20041
20042
20043
20044
20045
20046
20047
20048
20049
20050
20051
20052
20053
20054
20055
20056
20057
20058
20059
20060
20061
20062
20063
20064
20065
20066
20067
20068
20069
20070
20071
20072
20073
20074
20075
20076
20077
20078
20079
20080
20081
20082
20083
20084
20085
20086
20087
20088
20089
20090
20091
20092
20093
20094
20095
20096
20097
20098
20099
20100
20101
20102
20103
20104
20105
20106
20107
20108
20109
20110
20111
20112
20113
20114
20115
20116
20117
20118
20119
20120
20121
20122
20123
20124
20125
20126
20127
20128
20129
20130
20131
20132
20133
20134
20135
20136
20137
20138
20139
20140
20141
20142
20143
20144
20145
20146
20147
20148
20149
20150
20151
20152
20153
20154
20155
20156
20157
20158
20159
20160
20161
20162
20163
20164
20165
2016

21366
21367
21368
21369
21370
21371
21372
21373
21374
21375
21376
21377
21378
21379
21380
21381
21382
21383
21384
21385
21386
21387
21388
21389
21390
21391
21392
21393
21394
21395
21396
21397
21398
21399
21400
21401
21402
21403
21404
21405
21406
21407
21408
21409
21410
21411
21412
21413
21414
21415
21416
21417
21418
21419
21420
21421
21422
21423
21424
21425
21426
21427
21428
21429
21430
21431
21432
21433
21434
21435
21436
21437
21438
21439
21440
21441
21442
21443
21444
21445
21446
21447
21448
21449
21450
21451
21452
21453
21454
21455
21456
21457
21458
21459
21460
21461
21462
21463
21464
21465
21466
21467
21468
21469
21470
21471
21472
21473
21474
21475
21476
21477
21478
21479
21480
21481
21482
21483
21484
21485
21486
21487
21488
21489
21490
21491
21492
21493
21494
21495
21496
21497
21498
21499
21500
21501
21502
21503
21504
21505
21506
21507
21508
21509
21510
21511
21512
21513
21514
21515
21516
21517
21518
21519
21520
21521
21522
21523
21524
21525
21526
21527
21528
21529
21530
21531
2153

22732
22733
22734
22735
22736
22737
22738
22739
22740
22741
22742
22743
22744
22745
22746
22747
22748
22749
22750
22751
22752
22753
22754
22755
22756
22757
22758
22759
22760
22761
22762
22763
22764
22765
22766
22767
22768
22769
22770
22771
22772
22773
22774
22775
22776
22777
22778
22779
22780
22781
22782
22783
22784
22785
22786
22787
22788
22789
22790
22791
22792
22793
22794
22795
22796
22797
22798
22799
22800
22801
22802
22803
22804
22805
22806
22807
22808
22809
22810
22811
22812
22813
22814
22815
22816
22817
22818
22819
22820
22821
22822
22823
22824
22825
22826
22827
22828
22829
22830
22831
22832
22833
22834
22835
22836
22837
22838
22839
22840
22841
22842
22843
22844
22845
22846
22847
22848
22849
22850
22851
22852
22853
22854
22855
22856
22857
22858
22859
22860
22861
22862
22863
22864
22865
22866
22867
22868
22869
22870
22871
22872
22873
22874
22875
22876
22877
22878
22879
22880
22881
22882
22883
22884
22885
22886
22887
22888
22889
22890
22891
22892
22893
22894
22895
22896
22897
2289

24098
24099
24100
24101
24102
24103
24104
24105
24106
24107
24108
24109
24110
24111
24112
24113
24114
24115
24116
24117
24118
24119
24120
24121
24122
24123
24124
24125
24126
24127
24128
24129
24130
24131
24132
24133
24134
24135
24136
24137
24138
24139
24140
24141
24142
24143
24144
24145
24146
24147
24148
24149
24150
24151
24152
24153
24154
24155
24156
24157
24158
24159
24160
24161
24162
24163
24164
24165
24166
24167
24168
24169
24170
24171
24172
24173
24174
24175
24176
24177
24178
24179
24180
24181
24182
24183
24184
24185
24186
24187
24188
24189
24190
24191
24192
24193
24194
24195
24196
24197
24198
24199
24200
24201
24202
24203
24204
24205
24206
24207
24208
24209
24210
24211
24212
24213
24214
24215
24216
24217
24218
24219
24220
24221
24222
24223
24224
24225
24226
24227
24228
24229
24230
24231
24232
24233
24234
24235
24236
24237
24238
24239
24240
24241
24242
24243
24244
24245
24246
24247
24248
24249
24250
24251
24252
24253
24254
24255
24256
24257
24258
24259
24260
24261
24262
24263
2426

25464
25465
25466
25467
25468
25469
25470
25471
25472
25473
25474
25475
25476
25477
25478
25479
25480
25481
25482
25483
25484
25485
25486
25487
25488
25489
25490
25491
25492
25493
25494
25495
25496
25497
25498
25499
25500
25501
25502
25503
25504
25505
25506
25507
25508
25509
25510
25511
25512
25513
25514
25515
25516
25517
25518
25519
25520
25521
25522
25523
25524
25525
25526
25527
25528
25529
25530
25531
25532
25533
25534
25535
25536
25537
25538
25539
25540
25541
25542
25543
25544
25545
25546
25547
25548
25549
25550
25551
25552
25553
25554
25555
25556
25557
25558
25559
25560
25561
25562
25563
25564
25565
25566
25567
25568
25569
25570
25571
25572
25573
25574
25575
25576
25577
25578
25579
25580
25581
25582
25583
25584
25585
25586
25587
25588
25589
25590
25591
25592
25593
25594
25595
25596
25597
25598
25599
25600
25601
25602
25603
25604
25605
25606
25607
25608
25609
25610
25611
25612
25613
25614
25615
25616
25617
25618
25619
25620
25621
25622
25623
25624
25625
25626
25627
25628
25629
2563

IndexError: single positional indexer is out-of-bounds

In [13]:
final_df = pd.DataFrame(list(zip(first_name,last_name,User_Screen_Name,full_name)),columns=["First Name","Last Name","User Screen Name","Full Name"]).drop_duplicates()#.to_excel('Downloads/Match_Authors.xlsx')
final_df

First Name Last Name User Screen Name                Full Name
0           David    Butler          gorskon    David Gorski, MD, PhD
1           David    Butler   realDSteinberg          David Steinberg
2           David    Butler    DavidFinfrock           David Finfrock
3           David    Butler       davidsongc  Giles Davidson 🇺🇸🇺🇦🏳️‍🌈
4           David    Butler         DrDave01     David E. Albert, M.D
...           ...       ...              ...                      ...
434606    Jeffrey    Dunham      jeffreys819          Jeffrey Stevens
434607    Jeffrey    Dunham   JeffreyLucifer    Jeffrey McSorley🏳️‍🌈🎮
434608    Jeffrey    Dunham     thebadman3cf           Jeffrey Miller
434609    Jeffrey    Dunham     alootbrc2m8z             Jeffrey Hale
434610    Jeffrey    Dunham     JeffreyL2137                 Jeff2001

[432261 rows x 4 columns]

In [14]:
final_df['complete_name'] = final_df['First Name']+" "+final_df['Last Name']

In [15]:
final_df

First Name Last Name User Screen Name                Full Name  \
0           David    Butler          gorskon    David Gorski, MD, PhD   
1           David    Butler   realDSteinberg          David Steinberg   
2           David    Butler    DavidFinfrock           David Finfrock   
3           David    Butler       davidsongc  Giles Davidson 🇺🇸🇺🇦🏳️‍🌈   
4           David    Butler         DrDave01     David E. Albert, M.D   
...           ...       ...              ...                      ...   
434606    Jeffrey    Dunham      jeffreys819          Jeffrey Stevens   
434607    Jeffrey    Dunham   JeffreyLucifer    Jeffrey McSorley🏳️‍🌈🎮   
434608    Jeffrey    Dunham     thebadman3cf           Jeffrey Miller   
434609    Jeffrey    Dunham     alootbrc2m8z             Jeffrey Hale   
434610    Jeffrey    Dunham     JeffreyL2137                 Jeff2001   

         complete_name  
0         David Butler  
1         David Butler  
2         David Butler  
3         David Butler  
4         David Butler  
...                ...  
434606  Jeffrey Dunham  
434607  Jeffrey Dunham  
434608  Jeffrey Dunham  
434609  Jeffrey Dunham  
434610  Jeffrey Dunham  

[432261 rows x 5 columns]

In [16]:
from fuzzywuzzy import process
def get_closest_match(name, names_list):
    try:
        matches = process.extractOne(name, names_list)
        if matches[1]>70:
            return matches[0]
        else:
            return None
    except:
        return None

In [69]:
import re
my_list = [re.sub(' +', ' ', re.sub('[^a-zA-Z0-9.?]',' ',x).rstrip().lstrip()) for x in final_df['Full Name']]
my_list = [x for x in my_list if len(x.strip().split()) > 1]

In [90]:
data1['Full Name 1'] = data1['Full Name'].apply(lambda x: re.sub(' +', ' ', re.sub('[^a-zA-Z0-9.?]',' ',x).rstrip().lstrip()))

In [92]:
data1[['Full Name','Full Name 1','User Screen Name']].to_excel('Downloads/fghg.xlsx')

In [29]:
final_df['ClosestMatch'] = final_df['complete_name'].apply(lambda x: get_closest_match(x, final_df['Full Name']))

KeyboardInterrupt: 

In [48]:
final_df.to_excel('Downloads/Match_Authors.xlsx')

In [17]:
#Output
mapped_names_dict = {}
import re
for mapped_name in final_df['complete_name'].unique():
    # Get all 'FullName' values for the current 'MappedNames'
    full_names_list = final_df[final_df['complete_name'] == mapped_name]['Full Name'].tolist()
    # Add the list to the dictionary
    my_list = [re.sub(' +', ' ', re.sub('[^a-zA-Z0-9.?]',' ',x).rstrip().lstrip()) for x in full_names_list]
    my_list = [x for x in my_list if len(x.strip().split()) > 1]
    mapped_names_dict[mapped_name] = get_closest_match(mapped_name,my_list)

In [18]:
# Write Output
pd.DataFrame(mapped_names_dict.items(),columns=['Complete Name','Closest Name']).to_excel('Downloads/Veeva Match_Authors4.xlsx')

In [11]:
final_df1 = pd.DataFrame(mapped_names_dict.items(),columns=['Complete Name','Full Name'])
final_df1

Complete Name                          Full Name
0      Joseph Grossman        Joseph Kim MD MPH MBA FACHE
1         Kevin Rowley                           Kevin W.
2       Justin Asquith                          Justin W.
3        Muhammad Awan   Naba a Muhammad Richard Muhammad
4         Elias Feanny          Elias X escapee McClellan
...                ...                                ...
6730      Timothy Beer                Timothy J. McIntosh
6731      Barbara Rush      Carol RN Miss Rush the Gipper
6732       Lionel Chow                  Chow Ranch Studio
6733  Jennifer Prather  Jennifer Tseng IG tsengjennifermd
6734     Michael Bidus               Michael Fisch MD MPH

[6735 rows x 2 columns]